<h3>Genetic Algorithm</h3>

Genetic algorithms simulate the process of natural selection [ is a mechanism of evolution that explains how populations of living organisms adapt and change over time. ]. In simpler words, they simulate 'survival of the fittest' among individuals of consecutive generations to solve a problem.

<h5>Libraries and variables</h5>

In [1]:
import random
import math

from tqdm import trange,tqdm #visualizing the loops

GLOBAL_CHROMOSOME_LENGTH=2
GLOBAL_RANDINT_MIN=0
GLOBAL_RANDINT_MAX=10
GLOBAL_POP_LIMIT=40

GLOBAL_MUTATOR_CHANCE=1
GLOBAL_MUTATOR_RANGE=1000

GLOBAL_EPOCH_COUNT=1
GLOBAL_FITTING_DATA=[i for i in range(GLOBAL_CHROMOSOME_LENGTH)]

<h5>Gene Class Creation</h5>

In [2]:
class Gene:
    chromosomes=[]
    fitness_factor=GLOBAL_CHROMOSOME_LENGTH+1 #1 higher than max, default
    def __init__(self,rand_gene=True,eval=False,fitting_data=GLOBAL_FITTING_DATA): #constructor, 
        self.chromosomes=[]
        if rand_gene:
            self.create_random_gene()
        if eval:
            self.fitness_evaluation(fitting_data)
        pass    
    def create_random_gene(self): #initializes the gene based on chromosome length and randint range
        for i in range(GLOBAL_CHROMOSOME_LENGTH):
            self.chromosomes.append(random.randint(GLOBAL_RANDINT_MIN,GLOBAL_RANDINT_MAX))
        pass
    def crossover_with_gene(self,input_gene): #crosses over another gene, uniform crossover, includes mutation
        child=Gene(rand_gene=False)
        for i in range(GLOBAL_CHROMOSOME_LENGTH):
            if bool(random.getrandbits(1)):
                child.chromosomes.append(self.chromosomes[i])
            else:
                child.chromosomes.append(input_gene.chromosomes[i])
        child.random_mutator()
        return child
    def random_mutator(self): #imitates mutation based on mutation chances 
        if random.randint(1,GLOBAL_MUTATOR_RANGE)<=GLOBAL_MUTATOR_CHANCE:
            self.chromosomes[random.randint(0,GLOBAL_CHROMOSOME_LENGTH-1)]=random.randint(GLOBAL_RANDINT_MIN,GLOBAL_RANDINT_MAX)
    def fitness_evaluation(self,fitting_data): #generates the fitness factor based on fitting_data
        self.fitness_factor=GLOBAL_CHROMOSOME_LENGTH
        for i in range(GLOBAL_CHROMOSOME_LENGTH):
            if self.chromosomes[i]==fitting_data[i]:
                self.fitness_factor-=1
        return self.fitness_factor
    def display(self): #returnes a string with the chromosomes and fitness factor as output
        return str('Chromosomes: '+str(self.chromosomes)+' Fitness Factor: '+str(self.fitness_factor))


<h5>Gene List Class</h5>

In [3]:
class Gene_List: #WIP
    #initialization of lists
    fitting_data=[]
    generation_list=[]
    index_list=[]
    gene_list=[]
    
    def __init__(self,fitting_data=GLOBAL_FITTING_DATA):
        self.fitting_data=fitting_data
        self.generation_list=[]                         #emptying
        self.index_list=[]                              #emptying
        self.gene_list=self.Gene_List_Gen()             #generate initial gene list [Square]
        self.generation_list.append(self.gene_list)     #appends the initial gene list to the generation list, it is generation zero
        self.List_Iterator()                            #calls the auto iterator
        #self.generation_list.append(self.Gene_List_Cross())
        pass
    def List_Iterator(self): #Auto Iterator
        for i in trange(GLOBAL_EPOCH_COUNT,desc='Epoch'):
            if self.index_list==[]:                                                             #if no fit gene is found
                self.generation_list.append(self.Gene_List_Cross_I(self.generation_list[i]))    #generate new generation
                self.Gene_List_Eval_I(self.generation_list[i],stop=True)                        #evaluate the new generation
                if self.index_list!=[]:                                                         #if fit gene found
                    self.GLV(self.generation_list[i],self.index_list)                           #display fit gene
                    break                                                                       #end loop
    def Gene_List_Gen(self): #Gene List Generator [Initial] [Standalone]
        result=[]
        temp=[]
        for i in trange(GLOBAL_POP_LIMIT,desc='Generating genes'):
            temp=[]
            for j in range(GLOBAL_POP_LIMIT):
                temp.append(Gene(rand_gene=True,eval=True,fitting_data=self.fitting_data))
            result.append(temp)
        return result
    def Gene_List_Cross(self,method=0): #Gene List Crossover Function [Standalone]
        result=[]
        cross=[]
        temp=[]
        for i in trange(len(self.gene_list),desc='Crossover function executing'):
            for j in range(len(self.gene_list)):
                temp=[]
                for k in range(len(self.gene_list)):
                    for l in range(len(self.gene_list)):
                        temp.append(self.gene_list[i][j].crossover_with_gene(self.gene_list[k][l]))
                cross.append(temp)
        if method==0:
            rand_surv=[]
            for i in range(GLOBAL_POP_LIMIT):
                for j in range(GLOBAL_POP_LIMIT):
                    temp=[]
                    for k in range(2):
                        temp.append(random.randint(0,len(cross)-1))
                    rand_surv.append(temp)
            for i in trange(len(rand_surv),desc='Pruning crossover [Random]'):
                temp2=[]
                for x in range(40):
                    temp=[]
                    for j in range(2):
                        temp.append(cross[int(rand_surv[i][0])][int(rand_surv[i][1])])
                    temp2.append(temp)
            result.append(temp2)
        return result
    def Gene_List_Cross_I(self,input_list,method=0): #Gene List Crossover Function [Iterator Version]
        result=[]
        cross=[]
        temp=[]
        for i in trange(len(input_list),desc='Crossover function executing'):
            for j in range(len(input_list)):
                temp=[]
                for k in range(len(input_list)):
                    for l in range(len(input_list)):
                        temp.append(input_list[i][j].crossover_with_gene(input_list[k][l]))
                cross.append(temp)
        if method==0:
            result=cross
        return result
    def Gene_List_Eval(self,stop=False): #Gene List Fitness Evaluation Function [Standalone]
        result=[]
        temp=[]
        for i in trange(len(self.gene_list),desc='Evaluating genes'):
            for j in range(len(self.gene_list[0])):
                if self.gene_list[i][j].fitness_evaluation(self.fitting_data)==0:
                    temp=[]
                    temp.append(i)
                    temp.append(j)
                    result.append(temp)
                    if stop:
                        return result
        return result
    def Gene_List_Eval_I(self,input_list,stop=False): #Gene List Fitness Evaluation Function [Iterator Version]
        result=[]
        temp=[]
        for i in trange(len(input_list),desc='Evaluating genes'):
            for j in range(len(input_list[0])):
                if input_list[i][j].fitness_evaluation(self.fitting_data)==0:
                    temp=[]
                    temp.append(i)
                    temp.append(j)
                    result.append(temp)
                    if stop:
                        self.index_list=result
                        return
        self.index_list=result
        pass
    def Gene_List_Viewer(self): #Gene List Viewer [Standalone]
        for i in tqdm(self.index_list,desc='Displaying genes'):
            print('Gene '+str(i)+' '+self.gene_list[i[0]][i[1]].display())
    def GLV(self,input_list,index_list): #Gene List Viewer [Iterator Version]
        for i in tqdm(self.index_list,desc='Displaying genes'):
            print('Gene '+str(i)+' '+input_list[i[0]][i[1]].display())
    def GLCEF(self,input_list,stop=False): #Gene List Crossover & Evaluation Function [Iterator Version] [Generation Generator] [Deprecated]
        result=[]
        temp=[]
        for i in trange(len(input_list),desc='Crossover & Evaluation function executing'):
            for j in range(len(input_list)):
                temp=[]
                for k in range(len(input_list)):
                    for l in range(len(input_list)):
                        child=input_list[i][j].crossover_with_gene(input_list[k][l])
                        temp.append(child)
                        child.fitness_evaluation(self.fitting_data)
                        if child.fitness_factor==0:
                            temp2=[]
                            temp2.append(i)
                            temp2.append(j)
                            self.index_list.append(temp2)
                            if stop:
                                pass
                                
                result.append(temp)
        return result

In [4]:
GList=Gene_List()

Epoch:   0%|          | 0/1 [00:04<?, ?it/s]

Gene [0, 8] Chromosomes: [0, 1] Fitness Factor: 0


In [5]:
def GLG(): #Gene List Generator [Initial]
    result=[]
    temp=[]
    for i in trange(GLOBAL_POP_LIMIT,desc='Generating genes'):
        temp=[]
        for j in range(GLOBAL_POP_LIMIT):
            temp.append(Gene(rand_gene=True))
        result.append(temp)
    return result

In [6]:
def GLCF(input_list): #Gene List Crossover Function
    result=[]
    temp=[]
    for i in trange(len(input_list),desc='Crossover function executing'):
        for j in range(len(input_list)):
            temp=[]
            for k in range(len(input_list)):
                for l in range(len(input_list)):
                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l]))
            result.append(temp)
    return result

In [7]:
def GLFEF(input_list,fitting_data,stop=False): #Gene List Fitness Evaluation Function
    result=[]
    temp=[]
    for i in trange(len(input_list),desc='Evaluating genes'):
        for j in range(len(input_list[0])):
            if input_list[i][j].fitness_evaluation(fitting_data)==0:
                temp=[]
                temp.append(i)
                temp.append(j)
                result.append(temp)
                if stop:
                    return result
    return result

In [8]:
def GLV(input_list,index_list): #Gene List Viewer
    for i in tqdm(index_list,desc='Displaying genes'):
        print('Gene '+str(i)+' '+input_list[i[0]][i[1]].display())

In [9]:
def GCLRS(input_list): #Gene Crossover List Random Sampler
    len_list=len(input_list)
    parent_1=random.randint(0,len_list-1)
    parent_2=random.randint(0,len_list-1)
    print('\nParent 1 ['+str(parent_1)+']:'+input_list[parent_1][parent_1].display())
    print('\nParent 2 ['+str(parent_2)+']:'+input_list[parent_1][parent_1].display())
    print('\nChild 1 (P1xP2): '+input_list[parent_1][parent_2].display())
    print('\nChild 2 (P2xP1): '+input_list[parent_2][parent_1].display())

In [10]:
fitting_data=[x for x in range(GLOBAL_CHROMOSOME_LENGTH)]
list_of_genes=GLG()
first_gen=GLCF(list_of_genes)
index_list=GLFEF(first_gen,fitting_data=fitting_data,stop=True)
GLV(first_gen,index_list)

Displaying genes: 100%|██████████| 1/1 [00:00<?, ?it/s]

Gene [0, 16] Chromosomes: [0, 1] Fitness Factor: 0


In [11]:
run=True
gene_list=GLG()
generation_list=[]
generation_list.append(gene_list)
index_list=[]
while run==False:
    for i in trange(GLOBAL_ITERATION_LIMIT):
        if index_list==[]:
            generation_list.append(GLCF(generation_list[i]))
            index_list=GLFEF(generation_list[i],fitting_data,stop=True)
        if index_list!=[]:
            GLV(generation_list[i],index_list)
            run=False
            break
    run=False

Generating genes: 100%|██████████| 40/40 [00:00<00:00, 13330.06it/s]


In [12]:
GCLRS(first_gen)


Parent 1 [1586]:Chromosomes: [9, 5] Fitness Factor: 3

Parent 2 [49]:Chromosomes: [9, 5] Fitness Factor: 3

Child 1 (P1xP2): Chromosomes: [10, 5] Fitness Factor: 3

Child 2 (P2xP1): Chromosomes: [10, 5] Fitness Factor: 3


In [13]:
class Gene_ID_Gen: #deprecated
    gene_id_tracker=0
    def generate(self):
        gene_id=self.gene_id_tracker+1
        self.gene_id_tracker=self.gene_id_tracker+1
        return gene_id
    def latest(self):
        return self.gene_id_tracker

ID_Gen=Gene_ID_Gen()